## Carga de Datos 

In [10]:
import pandas as pd

import os
import numpy as np
import pandas as pd
from sklearn import tree
import sklearn as sklearn

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # for 3D plots
import seaborn as sns; sns.set()

df_kaggle = pd.read_csv("vgsales.csv")
print(df_kaggle.head())


   Rank                      Name Platform    Year         Genre Publisher  \
0     1                Wii Sports      Wii  2006.0        Sports  Nintendo   
1     2         Super Mario Bros.      NES  1985.0      Platform  Nintendo   
2     3            Mario Kart Wii      Wii  2008.0        Racing  Nintendo   
3     4         Wii Sports Resort      Wii  2009.0        Sports  Nintendo   
4     5  Pokemon Red/Pokemon Blue       GB  1996.0  Role-Playing  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0     41.49     29.02      3.77         8.46         82.74  
1     29.08      3.58      6.81         0.77         40.24  
2     15.85     12.88      3.79         3.31         35.82  
3     15.75     11.01      3.28         2.96         33.00  
4     11.27      8.89     10.22         1.00         31.37  


In [62]:
import os, time, math, requests, pandas as pd
from pandas import json_normalize

API_KEY = "601b475a2c3342c1860c7f432da840ca"          # <-- pon tu key
TARGET_N = 17000                # apunta un poco arriba de 16k por pérdidas al limpiar
PAGE_SIZE = 40                  # máximo permitido por RAWG
SLEEP_BASE = 0.8                # espera entre llamadas (ajústalo si ves 429)
OUT_EVERY = 1000                # guarda en disco cada N filas
OUT_CSV = "rawg_games_raw.csv"  # salida incremental
FINAL_CSV = "rawg_games_clean.csv"

def fetch_page(page: int, ordering="-added", dates=None):
    """
    ordering: ver docs RAWG; '-added' o '-rating' suelen devolver populares primero.
    dates: '2000-01-01,2025-12-31' para acotar (opcional).
    """
    params = {
        "key": API_KEY,
        "page": page,
        "page_size": PAGE_SIZE,
        "ordering": ordering
    }
    if dates:
        params["dates"] = dates

    # reintentos simples con backoff si hay 429/5xx
    for attempt in range(6):
        r = requests.get("https://api.rawg.io/api/games", params=params, timeout=30)
        if r.status_code == 200:
            return r.json()
        if r.status_code in (429, 500, 502, 503, 504):
            wait = SLEEP_BASE * (2 ** attempt)
            time.sleep(wait)
        else:
            r.raise_for_status()
    raise RuntimeError(f"Fallo permanente en página {page}: {r.status_code} {r.text[:200]}")

# --- Descarga paginada con guardado incremental ---
all_rows = []
seen_ids = set()
total_fetched = 0
page = 1

# Si ya existe un CSV previo, retoma desde ahí
if os.path.exists(OUT_CSV):
    prev = pd.read_csv(OUT_CSV)
    seen_ids = set(prev["id"].tolist())
    total_fetched = len(prev)
    print(f"Retomando: {total_fetched} filas ya guardadas.")
    all_rows = prev.to_dict(orient="records")

while total_fetched < TARGET_N:
    data = fetch_page(page, ordering="-added")   # puedes usar también "-rating" o "-metacritic"
    results = data.get("results", [])
    if not results:
        print("No hay más resultados (next=None).")
        break

    # filtra duplicados por id
    new = [row for row in results if row["id"] not in seen_ids]
    for row in new:
        seen_ids.add(row["id"])
        all_rows.append(row)

    total_fetched = len(all_rows)
    print(f"Página {page} | nuevos={len(new)} | total={total_fetched}")

    # guardado incremental
    if total_fetched % OUT_EVERY < PAGE_SIZE:  # aprox cada OUT_EVERY
        pd.DataFrame(all_rows).to_csv(OUT_CSV, index=False)
        print(f"Guardado parcial -> {OUT_CSV}")

    page += 1
    time.sleep(SLEEP_BASE)  # cortesía para no saturar

# guardado final crudo
pd.DataFrame(all_rows).to_csv(OUT_CSV, index=False)
print(f"Descarga cruda completa: {len(all_rows)} filas -> {OUT_CSV}")

# --- Normalización y limpieza (aplanar columnas útiles) ---
df = pd.read_csv(OUT_CSV)

# json_normalize maneja listas/dict si cargamos desde objetos; si cargaste desde csv,
# vuelve a leer desde 'all_rows' o re-normaliza antes de guardar. Aquí rehacemos:
df_norm = json_normalize(all_rows, max_level=1)

# columnas derivadas legibles
def join_list(objs, path_names):
    if isinstance(objs, list):
        try:
            if path_names == "platforms":
                return ", ".join(o["platform"]["name"] for o in objs)
            if path_names == "genres":
                return ", ".join(o["name"] for o in objs)
            if path_names == "stores":
                return ", ".join(o["store"]["name"] for o in objs)
        except Exception:
            return None
    return None

df_norm["platforms_names"] = df_norm["platforms"].apply(lambda x: join_list(x, "platforms"))
df_norm["genres_names"]     = df_norm["genres"].apply(lambda x: join_list(x, "genres"))
df_norm["stores_names"]     = df_norm["stores"].apply(lambda x: join_list(x, "stores"))

# selecciona columnas clave
cols = [
    "id","slug","name","released","rating","ratings_count","metacritic","playtime",
    "added","suggestions_count","updated","platforms_names","genres_names","stores_names",
    "background_image","esrb_rating.name"
]
cols = [c for c in cols if c in df_norm.columns]
df_rawg = df_norm[cols].copy()

# elimina duplicados (por id y por nombre por si acaso)
df_rawg = df_rawg.drop_duplicates(subset=["id"])
df_rawg = df_rawg.drop_duplicates(subset=["name"])  # opcional

# guarda limpio
df_rawg.to_csv(FINAL_CSV, index=False)
print(f"Limpio listo: {len(df_rawg)} filas -> {FINAL_CSV}")





Página 1 | nuevos=40 | total=40
Página 2 | nuevos=40 | total=80
Página 3 | nuevos=40 | total=120
Página 4 | nuevos=40 | total=160
Página 5 | nuevos=40 | total=200
Página 6 | nuevos=40 | total=240
Página 7 | nuevos=40 | total=280
Página 8 | nuevos=40 | total=320
Página 9 | nuevos=40 | total=360
Página 10 | nuevos=40 | total=400
Página 11 | nuevos=40 | total=440
Página 12 | nuevos=40 | total=480
Página 13 | nuevos=40 | total=520
Página 14 | nuevos=40 | total=560
Página 15 | nuevos=40 | total=600
Página 16 | nuevos=40 | total=640
Página 17 | nuevos=40 | total=680
Página 18 | nuevos=40 | total=720
Página 19 | nuevos=40 | total=760
Página 20 | nuevos=40 | total=800
Página 21 | nuevos=40 | total=840
Página 22 | nuevos=40 | total=880
Página 23 | nuevos=40 | total=920
Página 24 | nuevos=40 | total=960
Página 25 | nuevos=40 | total=1000
Guardado parcial -> rawg_games_raw.csv
Página 26 | nuevos=40 | total=1040
Página 27 | nuevos=40 | total=1080
Página 28 | nuevos=40 | total=1120
Página 29 | nuev

In [63]:
print("RAWG cols:", df_rawg.columns.tolist())
print(df_rawg[["name","platforms_names","genres_names","stores_names"]].head(3))
print(df_rawg[["platforms_names","genres_names","stores_names"]].isna().mean() * 100)


RAWG cols: ['id', 'slug', 'name', 'released', 'rating', 'ratings_count', 'metacritic', 'playtime', 'added', 'suggestions_count', 'updated', 'platforms_names', 'genres_names', 'stores_names', 'background_image', 'esrb_rating.name']
                       name  \
0        Grand Theft Auto V   
1  The Witcher 3: Wild Hunt   
2                  Portal 2   

                                     platforms_names     genres_names  \
0  PC, PlayStation 5, Xbox Series S/X, PlayStatio...           Action   
1  PlayStation 5, Xbox Series S/X, macOS, PlaySta...      Action, RPG   
2  PlayStation 3, PC, Xbox 360, Linux, macOS, Xbo...  Shooter, Puzzle   

                                        stores_names  
0  Steam, PlayStation Store, Epic Games, Xbox 360...  
1  GOG, PlayStation Store, Steam, Xbox Store, Nin...  
2  Xbox Store, Steam, PlayStation Store, Xbox 360...  
platforms_names    0.0
genres_names       0.0
stores_names       0.0
dtype: float64


In [ ]:
from rapidfuzz import process, fuzz
import pandas as pd
import re, unicodedata

# --- Función para normalizar nombres ---
def normalize_name(s: str) -> str | None:
    if pd.isna(s): 
        return None
    # quitar acentos/tildes
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii", "ignore").decode("ascii")
    # minúsculas
    s = s.lower()
    # eliminar texto entre paréntesis
    s = re.sub(r"\s*\(.*?\)", "", s)
    # eliminar palabras comunes irrelevantes
    s = re.sub(r"\b(edition|remastered|hd|ultimate|complete|definitive|remake|collection)\b", "", s)
    # quitar caracteres no alfanuméricos
    s = re.sub(r"[^a-z0-9]+", " ", s)
    return re.sub(r"\s+", " ", s).strip()

# --- Crear columnas normalizadas ---
df_kaggle["name_norm"] = df_kaggle["Name"].apply(normalize_name)
df_rawg["name_norm"]   = df_rawg["name"].apply(normalize_name)

# --- Matching usando la versión normalizada ---
rawg_names = df_rawg["name_norm"].tolist()
matches = []

for i, name in enumerate(df_kaggle["name_norm"]):
    match, score, idx = process.extractOne(name, rawg_names, scorer=fuzz.token_sort_ratio)

    kaggle_row = df_kaggle.iloc[i].to_dict()
    rawg_row   = df_rawg.iloc[idx].to_dict()

    combined = {**kaggle_row, **rawg_row, "match_score": score}
    matches.append(combined)

# Crear DataFrame final
df_matches = pd.DataFrame(matches)





In [83]:
# Guardar CSV
df_matches.to_csv("data.csv", index=False, encoding="utf-8")
print(df_matches.columns.tolist())

['Rank', 'Name', 'Platform', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'id', 'slug', 'released', 'rating', 'ratings_count', 'metacritic', 'playtime', 'added', 'suggestions_count', 'updated', 'platforms_names', 'genres_names', 'stores_names', 'background_image', 'esrb_rating.name', 'match_score']


## Exploración de Datos

In [82]:
#Completitud
df = df_matches
df.isnull().sum()

print ( df.isnull().sum()/len(df)*100)


Rank                  0.000000
Name                  0.000000
Platform              0.000000
Genre                 0.000000
Publisher             0.349440
NA_Sales              0.000000
EU_Sales              0.000000
JP_Sales              0.000000
Other_Sales           0.000000
Global_Sales          0.000000
id                    0.000000
slug                  0.000000
released              1.608628
rating                0.000000
ratings_count         0.000000
metacritic           54.289673
playtime              0.000000
added                 0.000000
suggestions_count     0.000000
updated               0.000000
platforms_names       0.000000
genres_names          0.000000
stores_names          0.000000
background_image      0.234968
esrb_rating.name     45.372936
match_score           0.000000
dtype: float64


Eliminar las columnas metacritic ya que más del 50% de los datos estan faltando por ende si se llegará a imputar el promedio o la media se generaría un sesgo sobre estos. Asimismo, se decide eliminar esrb_rating.name. Para el resto de faltantes se dicide eliminar las filas que no tengan algun dato, ya que la cantidad es mínima.

In [84]:
df.shape

(16598, 26)

In [85]:
#Unicidad
# 1) Duplicados exactos de filas completas
dup_mask_all = df_matches.duplicated(keep=False)
df_matches.loc[dup_mask_all].head()




,Rank,Name,Platform,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,playtime,added,suggestions_count,updated,platforms_names,genres_names,stores_names,background_image,esrb_rating.name,match_score


No hay duplicados por ende no es necesario eliminar ninguna fila.

In [86]:

print(df.dtypes)

Rank                   int64
Name                  object
Platform              object
Genre                 object
Publisher             object
NA_Sales             float64
EU_Sales             float64
JP_Sales             float64
Other_Sales          float64
Global_Sales         float64
id                     int64
slug                  object
released              object
rating               float64
ratings_count          int64
metacritic           float64
playtime               int64
added                  int64
suggestions_count      int64
updated               object
platforms_names       object
genres_names          object
stores_names          object
background_image      object
esrb_rating.name      object
match_score          float64
dtype: object


In [ ]:
import pandas as pd

# Asegura tipos seguros
yk = pd.to_numeric(df["Year_num"], errors="coerce").astype("Int64")

# Si ya tienes released_year, úsalo; si no, sácalo desde 'released'
if "released_year" in df.columns:
    yr = pd.to_numeric(df["released_year"], errors="coerce").astype("Int64")
else:
    yr = pd.to_datetime(df["released"], errors="coerce").dt.year.astype("Int64")

mask_both = yk.notna() & yr.notna()

mismatch_year = df.loc[mask_both & (yk != yr)]
pct = round(len(mismatch_year) / mask_both.sum() * 100, 2)

print("→ % de filas con ambos años y discrepancia:", pct, "%")
mismatch_year[["Name","Year_num","name","released","released_year"]].head(10)



→ % de filas con ambos años y discrepancia: nan %


C:\Users\Daniela\AppData\Local\Temp\ipykernel_10996\319936253.py:16: RuntimeWarning: invalid value encountered in scalar divide
  pct = round(len(mismatch_year) / mask_both.sum() * 100, 2)


,Name,Year_num,name,released,released_year
